In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
# Assuming that we are on a CUDA machine, this should print a CUDA device:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

def oprint(what, verbose):
    if verbose:
        print(what)

def imshow(img):
    print('originals')
    unimg = img / 2 + 0.5     # unnormalize
    npimg = unimg.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
    
def measures_plot(measures):
    plt.plot(measures['iteration_list'], measures['loss_list'])
    plt.xlabel("Number of iteration")
    plt.ylabel("Loss")
    plt.title("Loss")
    plt.show()

    plt.plot(measures['iteration_list'], measures['accuracy_list'], color = "red")
    plt.xlabel("Number of iteration")
    plt.ylabel("Accuracy")
    plt.title("Accuracy")
    plt.show()
    
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [ ]:
batch_size = 8

train_set = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

test_set = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
# get some random training images
dataiter = iter(train_loader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(batch_size)))

# to a great extent, the nn is tailored to the image size
print('image dimension for planning NN architecture')
print(images.size())

In [ ]:
class Net(nn.Module):
    def __init__(self, verbose = False):
        super(Net, self).__init__()
        
        self.verbose = verbose
        
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=2)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=4, stride=1, padding=0)
        self.conv3 = nn.Conv2d(32, 16, kernel_size=3, stride=1, padding=0)
        
        self.pool = nn.MaxPool2d(2)

        self.fc1 = nn.Linear(64, 48)
        self.fc2 = nn.Linear(48, 32)
        self.fc3 = nn.Linear(32, 10)
        
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        self.elu = nn.ELU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        oprint(x.size(), self.verbose)
        x = self.pool(self.relu(self.conv2(x)))
        oprint(x.size(), self.verbose)
        x = self.pool(self.elu(self.conv3(x)))
        oprint(x.size(), self.verbose)
        x = x.view(8, -1) # essential to be 8 channels (batch size)
        oprint(x.size(), self.verbose)
        x = self.tanh(self.fc1(x))
        oprint(x.size(), self.verbose)
        x = self.tanh(self.fc2(x))
        oprint(x.size(), self.verbose)
        x = self.fc3(x)
        oprint(x.size(), self.verbose)
        return x

model = Net(verbose = False)
model.to(device) # CUDA

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.3)

In [ ]:
count = 0
loss_list = []
iteration_list = []
accuracy_list = []

for epoch in range(10):  # loop over the dataset multiple times
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device) # CUDA
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        outputs = model(inputs)
        
        # Calculate gradients
        loss = criterion(outputs, labels)
        
        # back prop
        loss.backward()
        optimizer.step()
        
        count += 1

        # Prediction
        if count % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Predict test dataset
            for images, labels in test_loader:

                # Forward propagation
                outputs = model(images.to(device))

                # Get predictions from the maximum value
                predicted = torch.max(outputs.data, 1)[1]

                # Total number of labels
                total += len(labels)

                # Total correct predictions
                correct += (predicted == labels.to(device)).sum()

            accuracy = 100 * correct / float(total)
            loss_list.append(loss.data)
            iteration_list.append(count)
            accuracy_list.append(accuracy)

            if count % 2000 == 0:
                # Print Loss
                print('Epoch: {}  Iteration: {}  Loss: {}  Accuracy: {}%'.format(epoch,
                                                                                 count,
                                                                                 loss.data,
                                                                                 accuracy))
measures = {'loss_list' : loss_list,
           'accuracy_list' : accuracy_list,
           'iteration_list' : iteration_list}

measures_plot(measures)

In [ ]:
dataiter = iter(test_loader)
images, labels = dataiter.next()
outputs = model(images.to(device))
_, predicted = torch.max(outputs, 1)

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(batch_size)))
print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(batch_size)))

In [ ]:
correct = 0
total = 0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = model(images.to(device))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.to(device)).sum().item()
        c = (predicted == labels.to(device)).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].squeeze().item()
            class_total[label] += 1

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))
for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))